In [1]:
#!pip install wandb

In [2]:
#!pip install torchsummary

In [4]:
import warnings
warnings.filterwarnings('ignore')

In [5]:
import torch

print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0))

True
NVIDIA GeForce GTX 1080


In [6]:
import transformers

print(f"Running on transformers {transformers.__version__}")

Running on transformers 4.20.1


In [7]:
import wandb
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from collections import defaultdict
from datetime import datetime
from time import time
from datasets import load_dataset, load_metric
from pathlib import Path
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score, classification_report
from sklearn.model_selection import train_test_split, KFold
from torch.nn import BCEWithLogitsLoss
from torch.utils.data import Dataset, DataLoader
from tqdm.auto import tqdm
from transformers import (AdamW, 
                          AutoTokenizer, BertTokenizer, AlbertTokenizer, RobertaTokenizer, 
                          XLNetTokenizer, XLMRobertaTokenizer,
                          AlbertForSequenceClassification, AutoModelForSequenceClassification, 
                          DistilBertForSequenceClassification, BertForSequenceClassification,
                          XLNetForSequenceClassification, RobertaForSequenceClassification,
                          XLMRobertaForSequenceClassification,
                          get_scheduler, PreTrainedModel, 
                          DistilBertModel
                          )
from transformers.modeling_outputs import SequenceClassifierOutput
from matplotlib import pyplot as plt 

### load dataset

In [8]:
df = pd.read_json("au_2234_p.json")
df.head(10)

,text,labels,男性,女性,已婚,未婚
0,"我是不會去提醒啦,因為我是男生,如果對方是女生,我應該會默默觀賞就好!!","[1, 0, 0, 0]",1,0,0,0
1,我坐在旁邊看著打lol的男友「幹xxxx」「一群@#~?%」贏的時候，跩跩的輕哼一聲輸的時候...,"[0, 1, 0, 1]",0,1,0,1
2,人妻是我愛啊…只是除了道德上的問題還有太多現實上的問題，真的沒有勇氣去碰啊…,"[1, 0, 0, 0]",1,0,0,0
3,開學第一堂歷史課歷史老師說要重新選小老師我還在猶豫要不要舉手時一個男生舉手隨後我跟著舉手對兩...,"[0, 1, 0, 0]",0,1,0,0
4,(=′ー`)他們是很好的朋友她跟我男友固定聊天這樣不是對我男友有興趣吧？她是雙的樣子<br>,"[0, 1, 0, 1]",0,1,0,1
5,我不玩陣頭但是我有跟公司（就是俗稱的黑道我們公司算是比較正派的規定不准吃毒不准偷搶拐騙不找人...,"[1, 0, 0, 0]",1,0,0,0
6,我男友都叫我不要喝冰的但我跟他見面我還是盧洨成功喝冰的但他說會痛不要跟他說嗚嗚嗚ʕ´•ᴥ•̥`ʔ,"[0, 1, 0, 1]",0,1,0,1
7,"有些事情真的是習慣,不容易改啊~ 像我家老爺和孩子都是屬於過敏體質,揉口鼻這行為幾乎都快變成...","[0, 1, 1, 0]",0,1,1,0
8,太爽了，希望有一天我能突然大學教室，直接抓著正妹開幹,"[1, 0, 0, 0]",1,0,0,0
9,偶而我會和我老婆去吃到飽餐廳.不過我們只會吃到飽.不會吃到撐.吃的量也不是很多.只是抱著品嚐...,"[1, 0, 1, 0]",1,0,1,0


### data preprocessing

In [9]:
# label_col = ['男性', '女性', '已婚', '未婚', '上班族', '學生', '青年', '有子女']
label_col = ['男性', '女性', '已婚', '未婚']
len(label_col)
# num_idx = list(range(0,4))

# label_col_dict = dict(zip(label_col, num_idx))

# print(label_col)
# print(num_idx)
# print(label_col_dict)

4

In [ ]:
# def label_transform(df, label_col_dict, target):
#     one_hot_list = []
#     for labels in df[target]:
#         temp = np.zeros(8).astype(int)
#         for label in labels:
#             temp[label_col_dict.get(label)] = 1
#         one_hot_list.append(temp)
        
#     df['labels'] = one_hot_list
#     return df

In [10]:
def one_hot_to_label(arr, label_col):
    temp = []
    for i in range(len(arr)):
        if arr[i] == 1:
            temp.append(label_col[i])
    return temp

In [ ]:
# df = label_transform(
#     df=df, 
#     label_col_dict=label_col_dict,
#     target='label'
# )

In [11]:
sum_label = df[label_col].eq(1).sum()
sum_label

男性    1035
女性    1074
已婚    1066
未婚    1040
dtype: int64

In [12]:
print('max sentence length: ', max(len_list))
print('average sentence length: ', np.mean(len_list))
# print('stdev sentence length: ', df.text.str.split().str.len().std())
print("sample size: ", len(len_list))

NameError: name 'len_list' is not defined

In [13]:
class AudienceDataset(Dataset):
    def __init__(self, reviews, targets, tokenizer, max_len):
        self.reviews = reviews
        self.targets = targets
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __getitem__(self, idx):
        review = str(self.reviews[idx])
        target = self.targets[idx]
        encoding = self.tokenizer.encode_plus(
            review,
            add_special_tokens=True,
            max_length=self.max_len,
            return_token_type_ids=False,
            padding='max_length',
            return_attention_mask=True,
            return_tensors='pt',
            truncation=True
        )
        return {
          'review_text': review,
          'input_ids': encoding['input_ids'].flatten(),
          'attention_mask': encoding['attention_mask'].flatten(),
          'targets': torch.tensor(target)
        }
    def __len__(self):
        return len(self.reviews)

In [14]:
def create_data_loader(df, tokenizer, max_len, batch_size):
    ds = AudienceDataset(
        reviews=df.text.to_numpy(),
        targets=df.labels.to_numpy(),
        tokenizer=tokenizer,
        max_len=max_len
        )
    return DataLoader(
        ds,
        batch_size=batch_size,
        num_workers=0,
        shuffle= True,
        )

In [15]:
num_labels=len(label_col)
num_labels

4

### modeling

In [16]:
def accuracy_thresh(y_pred, y_true, thresh=0.5, average='weighted'):
    """
    :param y_pred: the sigmoid predictions of the model output, expect array
    :param y_true: the true labels array
    :param thresh: the threshold of multi-label classification, set default to 0.5
    :param average: the average method of metrics, set default to weighted
    :return: dictionary of accuracy, precision, recall and f1_score
    """
    y_pred = torch.from_numpy(y_pred)
    y_true = torch.from_numpy(y_true)

    a = accuracy_score(
        y_true=np.array(y_true.bool()),
        y_pred=np.array(y_pred > thresh)
    )

    p = precision_score(
        y_true=np.array(y_true.bool()),
        y_pred=np.array(y_pred > thresh),
        average=average
    )
    r = recall_score(
        y_true=np.array(y_true.bool()),
        y_pred=np.array(y_pred > thresh),
        average=average
    )
    f = f1_score(
        y_true=np.array(y_true.bool()),
        y_pred=np.array(y_pred > thresh),
        average=average
    )

    return {
        'accuracy': a * 100,
        'precision': p * 100,
        'recall': r * 100,
        'f1': f * 100
    }


In [17]:
def train_epoch(model, loader, optimizer, device, scheduler, loss_func=None):
    start = time()
    model = model.train()
    pred_list = []
    label = []
    for d in loader:
        optimizer.zero_grad()
        input_ids = d["input_ids"].to(device)
        attention_mask = d["attention_mask"].to(device)
        targets = d["targets"].to(device)
        outputs = model(
            input_ids=input_ids, 
            attention_mask=attention_mask,
            labels=targets.float()
        )
        
        logits = outputs.logits
        if loss_func:
            loss = loss_func(
                logits.view(-1,num_labels), 
                targets.float().view(-1, num_labels)
            )
        else:
            loss = outputs.loss
        
        
        pred_list.append(logits.sigmoid())
        label.append(targets)
        
        loss.backward()
        optimizer.step()
        scheduler.step()
        
    y_pred = np.array([l for b in pred_list for l in b.cpu().detach().numpy()])
    y_true = np.array([l for b in label for l in b.cpu().detach().numpy()])
    
    acc = accuracy_thresh(
        y_pred=y_pred,
        y_true=y_true,
        thresh=0.5
    )
    return loss, acc, (time() - start) / 60

In [18]:
def eval_epoch(model, loader, optimizer, device, scheduler, loss_func=None):
    start = time()
    model.eval()
    pred_list = []
    label = []
    for d in loader:
        input_ids = d["input_ids"].to(device)
        attention_mask = d["attention_mask"].to(device)
        targets = d["targets"].to(device)
        with torch.no_grad():
            outputs = model(
                input_ids=input_ids, 
                attention_mask=attention_mask,
                labels=targets.float()
            )

        logits = outputs.logits
        if loss_func:
            loss = loss_func(
                logits.view(-1,num_labels), 
                targets.float().view(-1, num_labels)
            )
        else:
            loss = outputs.loss
        
        pred_list.append(logits.sigmoid())
        label.append(targets)
    
    y_pred = np.array([l for b in pred_list for l in b.cpu().detach().numpy()])
    y_true = np.array([l for b in label for l in b.cpu().detach().numpy()])
    
    acc = accuracy_thresh(
        y_pred=y_pred,
        y_true=y_true,
        thresh=0.5,
        average="macro"
    )
    return loss, acc, (time() - start) / 60

In [19]:
def sigmoid_logits_to_one_hot(arr: np.array, thresh=0.5):
    arr[arr > thresh] = 1
    arr[arr <= thresh] = 0
    return arr.astype(int)

In [20]:
def get_prediction(model, data_loader, path, device):
    model.load_state_dict(torch.load(path))
    model = model.eval()
    review_texts = []
    predictions = []
    # prediction_probability = []
    real_values = []
    with torch.no_grad():
        for d in data_loader:
            texts = d['review_text']
            input_ids = d['input_ids'].to(device)
            attention_mask = d["attention_mask"].to(device)
            targets = d['targets'].to(device)
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            logits = outputs.logits
            review_texts.extend(texts)
            predictions.extend(logits.sigmoid())
            real_values.extend(targets)

        y_pred = np.array([b.cpu().detach().numpy() for b in predictions])
        y_true = np.array([b.cpu().detach().numpy() for b in real_values])
        
        y_pred = sigmoid_logits_to_one_hot(y_pred)
        # predictions = torch.stack(predictions).cpu().detach()
        # real_values = torch.stack(real_values).cpu().detach()
    return review_texts, y_pred, y_true

In [21]:
def get_classification_report(model, test_data_loader, path, device, label_col):
    y_review_texts, y_pred, y_test = get_prediction(model, test_data_loader, path, device)
    report = classification_report(y_test, y_pred, target_names=label_col, output_dict=True)
    report = pd.DataFrame(report).transpose()
    report = report.reset_index().rename(columns={'index':'label'})
    false_prediction = [['text', 'label', 'predict']]
    for i in range(len(y_pred)):
        if not (y_pred[i] == y_test[i]).all():
            false_prediction.append(
                [
                    y_review_texts[i], 
                    one_hot_to_label(y_test[i], label_col), 
                    one_hot_to_label(y_pred[i], label_col)
                ]
            )
    pd.set_option('display.float_format', lambda x: '%.3f' % x)
    false_prediction_df = pd.DataFrame(false_prediction[1:], columns=false_prediction[0])
    
    return report, false_prediction_df

In [22]:
# https://discuss.pytorch.org/t/finding-model-size/130275/2
def get_model_size(model):
    param_size = 0
    param_sum = 0
    for param in model.parameters():
        param_size += param.nelement() * param.element_size()
        param_sum += param.nelement()
        
    buffer_size = 0
    buffer_sum = 0
    for buffer in model.buffers():
        buffer_size += buffer.nelement() * buffer.element_size()
        buffer_sum += buffer.nelement()

    size_all_mb = (param_size + buffer_size) / 1024**2
    size_dict = {
        'param_size': param_size, 
        'param_sum': param_sum, 
        'buffer_size': buffer_size,
        'buffer_sum': buffer_sum,
        'total_size(MB)': size_all_mb
    }
    pd.set_option('display.float_format', lambda x: '%.3f' % x)
    size_df = pd.DataFrame(list(size_dict.items()), columns=['name', 'value'])
    size_df = size_df.round(2)
    return size_df

In [36]:
# https://pytorch.org/tutorials/beginner/saving_loading_models.html#export-load-model-in-torchscript-format
def save_pt(model, state_dict_path, pt_output_path):
    model.load_state_dict(torch.load(state_dict_path))
    model_scripted = torch.jit.script(model)
    model_scripted.save(f"{pt_output_path}.pt")

In [34]:
def run(
    df_train,
    df_test,
    learning_rate = 2e-5, 
    epochs = 50,
    batch_size = 32,
    max_len = 30,
    num_labels = 4,
    ckpt = "roberta-base",
    dsn = "au_1200_p",
    project_name = "audience_bert_4_class",
    prefix=None
):
    
    display_name = f"{ckpt}_{prefix}_{datetime.now()}" if prefix else f"{ckpt}_{datetime.now()}"
    wandb.init(project=project_name, name=display_name)
    config = {
        "dataset": dsn,
        "learning_rate": learning_rate,
        "epochs": epochs,
        "batch_size": batch_size,
        "max_len": max_len,
        "pre-trained model": ckpt,
    }

    wandb.config.update(config)

    tokenizer = AutoTokenizer.from_pretrained(ckpt, do_lower_case=True)
    model = AutoModelForSequenceClassification.from_pretrained(
        ckpt, num_labels=num_labels
    )

    model.config.problem_type = "multi_label_classification"

    train_loader = create_data_loader(df_train, tokenizer, max_len, batch_size)
    test_loader = create_data_loader(df_test, tokenizer, max_len, batch_size)

    optimizer = AdamW(model.parameters(), lr=learning_rate, correct_bias= False)

    num_epochs = epochs
    num_training_steps = num_epochs * len(train_loader)

    lr_scheduler = get_scheduler(
        name='linear', optimizer=optimizer,
        num_warmup_steps=0, num_training_steps=num_training_steps
    )

    device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
    model.to(device)

    progress_bar = tqdm(range(num_epochs))

    b_history = defaultdict(list)

    print(' *** Start training flow *** ')
    best_acc = 0
    best_epoch = 0

    wandb.watch(model, log="all")
    for epoch in range(1, num_epochs+1):
        print('='*100)
        print(f"Epoch {epoch} / {num_epochs}")

        print('-'*100)
        train_loss, train_acc, train_time = train_epoch(
            model, train_loader, optimizer, device, lr_scheduler, 
            loss_func=BCEWithLogitsLoss()
        )
        print(f"training loss {train_loss}; training time {train_time}")

        val_loss, val_acc, val_time = eval_epoch(
            model, test_loader, optimizer, device, lr_scheduler,
            loss_func=BCEWithLogitsLoss()
        )
        print(f"validation loss {val_loss}; validation time {val_time}")

        print(f"training acc {train_acc}")
        print(f"validation acc {val_acc}")

        if val_acc['f1'] > best_acc:
            best_acc = val_acc['f1']
            best_epoch = epoch
            torch.save(model.state_dict(), f"./model/{display_name.split('/')[-1]}_{dsn}.bin")
            wandb.save(f"{ckpt.split('/')[-1]}_{dsn}_{best_acc}.bin")
            print(f"best f1_score is updated: {best_acc}")

        b_history['train_acc'].append(train_acc)
        b_history['train_loss'].append(train_loss)
        b_history['val_acc'].append(val_acc)
        b_history['val_loss'].append(val_loss)

        wandb.log(
            {
                "epoch": epoch,
                "train/loss": train_loss,
                "train/accuracy": train_acc['f1'],
                "val/loss": val_loss,
                "val/accuracy": val_acc['f1'],
            }
        )

        progress_bar.update(1)
    print(' *** training is done !! *** ')
    print(f"Best epoch: {best_epoch}")
    print(f"Best f1_score: {best_acc}")

    report, false_prediction_df = get_classification_report(
        model, 
        test_loader, 
        f"./model/{display_name.split('/')[-1]}_{dsn}.bin",  
        device, 
        label_col
    )
    
    save_pt(
        model, 
        f"./model/{display_name.split('/')[-1]}_{dsn}.bin", 
        f"./model/torch_script/{ckpt}"
    )

    print(report)

    model_size = get_model_size(model)

    print(model_size)
    
    wandb.log({
        "classification_report": wandb.Table(dataframe=report),
        "false_prediction": wandb.Table(dataframe=false_prediction_df),
        "model_size_info": wandb.Table(dataframe=model_size)
    })

    wandb.finish()
    
    false_prediction_df.to_csv(f"./false_pred/{display_name.split('/')[-1]}.csv", encoding='utf-8', index=False)


### execute experiments

In [25]:
df_train, df_test = train_test_split(df, test_size=0.2, random_state = 42)
print(df_train.shape, df_test.shape)

(1787, 6) (447, 6)


In [ ]:
torch.cuda.empty_cache()

run(
    df_train=df_train, 
    df_test=df_test, 
    epochs = 3,
    batch_size=32, 
    max_len=100, 
    ckpt="hfl/chinese-roberta-wwm-ext"
)

epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
train/accuracy,▁▆▇▇████████████████████████████████████
train/loss,█▃▃▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val/accuracy,▃▄▁▄█▃▃▆▆▄▆▅▅▄▅▄▅▅▅▆▆▅▅▅▅▅▅▆▇▆▆▆▆▆▆▆▆▆▆▆
val/loss,▃▂▁█▃▁▃▂▂▅▅▃▇▂▃▄▃▃▆▃▆▄▄▄▆▄▄▂▅▄▃▃▄▃▄▁▅▄▁▄
epoch,50
train/accuracy,100.0
train/loss,0.00143
val/accuracy,95.15623
val/loss,0.22397


In [28]:
torch.cuda.empty_cache()

run(
    df_train=df_train, 
    df_test=df_test, 
    batch_size=32, 
    max_len=100, 
    ckpt="hfl/chinese-macbert-base"
)

Downloading:   0%|          | 0.00/19.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/659 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/107k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/263k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/393M [00:00<?, ?B/s]

Some weights of the model checkpoint at hfl/chinese-macbert-base were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not i

  0%|          | 0/50 [00:00<?, ?it/s]

 *** Start training flow *** 
Epoch 1 / 50
----------------------------------------------------------------------------------------------------
training loss 0.152379110455513; training time 0.3513642907142639
validation loss 0.2144685536623001; validation time 0.029507593313852946
training acc {'accuracy': 75.15388919977616, 'precision': 88.8794966621917, 'recall': 87.54448398576513, 'f1': 88.20138473572123}
validation acc {'accuracy': 84.78747203579418, 'precision': 93.12958420406443, 'recall': 93.78453105234743, 'f1': 93.32786890818099}
best f1_score is updated: 93.32786890818099
Epoch 2 / 50
----------------------------------------------------------------------------------------------------
training loss 0.1352216750383377; training time 0.3545873165130615
validation loss 0.18024364113807678; validation time 0.029735279083251954
training acc {'accuracy': 89.03189703413543, 'precision': 95.01820279297485, 'recall': 95.43297746144721, 'f1': 95.21683430356588}
validation acc {'accurac

epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
train/accuracy,▁▅▆▇████████████████████████████████████
train/loss,█▇▇▃▅▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val/accuracy,▁▂▆▇▇█▇▇▇▇▇▇▇▇█▇▇▇▇▇▆▇▆▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇
val/loss,▄▄▃▂▄▃▅▅▅▃▅▇▄▄▂▄▃▄█▁▃▃▁▁▇▃▅▄▄▆▅▁▅▅▄▃▅▅▄▄
epoch,50
train/accuracy,100.0
train/loss,0.00112
val/accuracy,94.97551
val/loss,0.20957


In [29]:
torch.cuda.empty_cache()

run(
    df_train=df_train, 
    df_test=df_test, 
    batch_size=32, 
    max_len=100, 
    ckpt="hfl/chinese-bert-wwm-ext"
)

Downloading:   0%|          | 0.00/19.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/647 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/107k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/263k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/393M [00:00<?, ?B/s]

Some weights of the model checkpoint at hfl/chinese-bert-wwm-ext were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkp

  0%|          | 0/50 [00:00<?, ?it/s]

 *** Start training flow *** 
Epoch 1 / 50
----------------------------------------------------------------------------------------------------
training loss 0.24103447794914246; training time 0.3552232265472412
validation loss 0.1728965789079666; validation time 0.029775754610697428
training acc {'accuracy': 61.55567991046447, 'precision': 81.38146289626853, 'recall': 80.93119810201661, 'f1': 81.1125215243812}
validation acc {'accuracy': 83.44519015659955, 'precision': 91.46473875303597, 'recall': 93.80440631002666, 'f1': 92.50378436028899}
best f1_score is updated: 92.50378436028899
Epoch 2 / 50
----------------------------------------------------------------------------------------------------
training loss 0.25974413752555847; training time 0.358618160088857
validation loss 0.09004504978656769; validation time 0.03005201816558838
training acc {'accuracy': 85.22663682148853, 'precision': 93.21780057411166, 'recall': 94.15776986951364, 'f1': 93.67651691501835}
validation acc {'accura

epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
train/accuracy,▁▆▇▇▇███████████████████████████████████
train/loss,▇█▃▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val/accuracy,▁▅▆▆▇▇▇▇███▇█▇█▇█▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇███▇▇▇▇▇
val/loss,▄▂▃▅▄▂▁▄▆▆▆▅▇▄▄▃▄▂▂▃█▂▂▂▄▁▅▆█▂▄▁▆▇▆▄▅▂▅▄
epoch,50
train/accuracy,100.0
train/loss,0.00147
val/accuracy,95.53786
val/loss,0.19147


In [30]:
torch.cuda.empty_cache()

run(
    df_train=df_train, 
    df_test=df_test, 
    batch_size=32, 
    max_len=100, 
    ckpt="bert-base-chinese"
)

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/624 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/107k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/263k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/393M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-chinese were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

  0%|          | 0/50 [00:00<?, ?it/s]

 *** Start training flow *** 
Epoch 1 / 50
----------------------------------------------------------------------------------------------------
training loss 0.19530321657657623; training time 0.35036675532658895
validation loss 0.3122142553329468; validation time 0.02926221291224162
training acc {'accuracy': 60.15668718522663, 'precision': 81.49640589415483, 'recall': 81.31672597864768, 'f1': 80.89374613833027}
validation acc {'accuracy': 80.53691275167785, 'precision': 91.27988126238525, 'recall': 91.85264518896491, 'f1': 91.51141599679725}
best f1_score is updated: 91.51141599679725
Epoch 2 / 50
----------------------------------------------------------------------------------------------------
training loss 0.09669455885887146; training time 0.3532953222592672
validation loss 0.08566690236330032; validation time 0.029499534765879312
training acc {'accuracy': 83.65976496922217, 'precision': 92.70016106684336, 'recall': 93.03084223013049, 'f1': 92.84729856378705}
validation acc {'acc

epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
train/accuracy,▁▅▆▇▇███████████████████████████████████
train/loss,▆▃▅█▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val/accuracy,▁▄▆▆▇███▇████▇▇███▆▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇
val/loss,▆▂▄▄▂▅▇▆▃▅▂▃▄▃▂▂▅█▄▄▄▄▄▂▇▄▁▆▇▅▄▅▄▂▄▆▅▇▅▆
epoch,50
train/accuracy,100.0
train/loss,0.00172
val/accuracy,95.19442
val/loss,0.32508


### K-fold cross validation

In [ ]:
kf = KFold(n_splits=5)
kf.get_n_splits(df.text.values)

In [ ]:
kf

In [ ]:
fold = 1
for train_idx, test_idx in tqdm(kf.split(df.text.values)):
    print("Fold number: ", fold)
    df_train = df.iloc[train_idx]
    df_test = df.iloc[test_idx]
    print("Size of datasets: ")
    print(df_train.shape, df_test.shape)
    
    run(
        df_train=df_train, 
        df_test=df_test, 
        batch_size=32, 
        max_len=64, 
        ckpt="bert-base-chinese",
        prefix=fold
    )
    # start training and validating
    fold += 1

In [ ]:
len(df)

### baseline

In [ ]:
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.multioutput import ClassifierChain
# from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
# def convert_feature(examples,
#                     max_features=50, 
#                     min_df=2, 
#                     vec=None
#                    ):
#     seg_contents = examples
#     if vec:
#         x_features = vec.transform(seg_contents)
#         return x_features
#     else:
#         vectorizer = TfidfVectorizer(max_features=max_features, min_df=min_df)
#         x_features = vectorizer.fit_transform(seg_contents)
#         return x_features, vectorizer

# def get_multi_accuracy(y_true, y_pre):
#     right = 0
#     wrong = 0
#     for i in range(len(y_true)):
#         for j in range(len(y_true[i])):
#             if y_true[i][j] == y_pre[i][j]:
#                 right += 1
#             else:
#                 wrong += 1
#     return right / (right + wrong)

In [ ]:
# X_train = df_train.text
# X_test = df_test.text

In [ ]:
# x_train_features, x_vec = convert_feature(X_train)
# x_test_features = convert_feature(X_test, vec=x_vec)

In [ ]:
# print(f"shape of training feature {x_train_features.shape}")
# print(f"shape of testing feature {x_test_features.shape}")

In [ ]:
# y_train = df_train[label_col].values
# y_test = df_test[label_col].values

In [ ]:
# %%time
# project_name = "audience_bert"
# display_name = f"baseline_RF_1"

# wandb.init(project=project_name, name=display_name)

# clf = ClassifierChain(RandomForestClassifier()).fit(x_train_features, y_train)
# pred = clf.predict(x_test_features)
# # acc = get_multi_accuracy(y_test, pred)
# # print(f"acc: {acc*100}")
# output = pd.DataFrame(
#     classification_report(y_test, pred, output_dict=True, target_names=label_col)
# ).transpose()
# output = output.reset_index().rename(columns={'index':'label'})
# print(output)

# wandb.log({
#     "classification_report": wandb.Table(dataframe=output),
# })

# wandb.finish()